# Example of generating summaries for a 10K
In this example, we will show you how to generate page summaries from a pdf using OpenAI's models via `uniflow`'s [OpenAIJsonModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L125).

For this example, we're using a [10K from Nike](https://investors.nike.com/investors/news-events-and-reports/).

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

Next, you will need a valid [OpenAI API key](https://platform.openai.com/api-keys) to run the code. Once you have the key, set it as the environment variable `OPENAI_API_KEY` within a `.env` file in the root directory of this repository. For more details, see this [instruction](https://github.com/CambioML/uniflow/tree/main#api-keys)

Finally, we are storing the Nike 10K in the `data\raw_input` directory as "nike-10k-2023.pdf". You can download the file from [here](https://s1.q4cdn.com/806093406/files/doc_downloads/2023/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf).

### Update system path

In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Install helper packages

In [2]:
!{sys.executable} -m pip install langchain pandas pypdf

### Import dependencies

In [3]:
from dotenv import load_dotenv
import os
import pandas as pd
from uniflow.transform.client import Client
from uniflow.transform.config import TransformOpenAIConfig
from uniflow.model.config import OpenAIModelConfig
from langchain.document_loaders import PyPDFLoader
from uniflow.schema import Context, GuidedPrompt

load_dotenv()


/Users/joseortiz/anaconda3/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Prepare the input data
First, we need to pre-process the PDF to get text chunks that we can feed into the model. We will use `PyPDFLoader` from langchain.

In [4]:
pdf_file = "nike-10k-2023.pdf"

##### Set current directory and input data directory.

In [5]:
dir_cur = os.getcwd()
input_file = os.path.join(f"{dir_cur}/data/raw_input/", pdf_file)

##### Load and split the pdf

In [6]:
loader = PyPDFLoader(input_file)
pages = loader.load_and_split()
page_contents = [page.page_content for page in pages]

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM. Because we are not generating the default questions and answers, we need to have a custom `instruction` and custom `examples`, which we configure in the `GuidedPrompt` class.

First, we give a custom `instruction` to the `GuidedPrompt`. This ensures we are instructing the LLM to generate summaries instead of the default questions and answers.

Next, we give a sample list of `Context` examples to the `GuidedPrompt` class. We pass in a custom `summary` property into our `Context` objects. This is an example summary based on the `context`.

In [7]:
guided_prompt = GuidedPrompt(
    instruction="Generate a one sentence summary based on the last context below. Follow the format of the examples below to include context and summary in the response",
    examples=[
        Context(
            context="When you're operating on the maker's schedule, meetings are a disaster. A single meeting can blow a whole afternoon, by breaking it into two pieces each too small to do anything hard in. Plus you have to remember to go to the meeting. That's no problem for someone on the manager's schedule. There's always something coming on the next hour; the only question is what. But when someone on the maker's schedule has a meeting, they have to think about it.",
            summary="Meetings disrupt the productivity of those following a maker's schedule, dividing their time into impractical segments, while those on a manager's schedule are accustomed to a continuous flow of tasks.",
        ),
    ],
)

Next, for the given `page_contents` above, we convert them to the `Context` class to be processed by `uniflow`.

In [8]:
data = [ Context(context=p[:800], summary="") for p in page_contents[6:16] if len(p) > 200 ]
data

[Context(context='We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third fiscal quarters.  However, the mix of product sales may vary \nconsiderably as a result of changes in seasonal and geographic demand for particular types of footwear , apparel and equipment, \nas well as other macroeconomic, strategic, operating and logistics-related factors.\nBecause NIKE is a consume', summary=''),
 Context(context="INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared t

### Use LLM to generate data

In this example, we will use the [OpenAIModelConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/model/config.py#L17)'s default LLM to generate questions and answers.

Here, we pass in our `guided_prompt` to the `TransformOpenAIConfig` to use our customized instructions and examples, instead of the `uniflow` default ones.

We also want to get the response in the `json` format instead of the `text` default, so we set the `response_format` to `json_object`.

In [9]:
config = TransformOpenAIConfig(
    guided_prompt_template=guided_prompt,
    model_config=OpenAIModelConfig(response_format={"type": "json_object"}),
)
client = Client(config)

Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.

In [10]:
output = client.run(data)

100%|██████████| 10/10 [00:34<00:00,  3.46s/it]


### Process the output

Let's take a look of the generated output. We need to do a little postprocessing on the raw output.

In [11]:
# Extracting context, question, and answer into a DataFrame
contexts = []
summaries = []

for item in output:
    for i in item.get('output', []):
        for response in i.get('response', []):
            if any(key not in response for key in ['context', 'summary']):
                print("Missing context or summary in response:", response)
                continue
            contexts.append(response['context'])
            summaries.append(response['summary'])

# Set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

df = pd.DataFrame({
    'Context': contexts,
    'Summaries': summaries,
})

df

,Context,Summaries
0,"We also offer interactive consumer services and experiences as well as digital products through our digital platforms, including \nfitness and activity apps; sport, fitness and wellness content; and digital services and features in retail stores that enhance the \nconsumer experience.\nSALES AND MARKETING\nWe experience moderate fluctuations in aggregate sales volume during the year. Historically, revenues in the first and fourth \nfiscal quarters have slightly exceeded those in the second and third fiscal quarters. However, the mix of product sales may vary \nconsiderably as a result of changes in seasonal and geographic demand for particular types of footwear , apparel and equipment, \nas well as other macroeconomic, strategic, operating and logistics-related factors.\nBecause NIKE is a consume","Nike offers interactive consumer services, digital products, and experiences through digital platforms, with moderate fluctuations in sales volume throughout the year based on seasonal and geographic demand."
1,"INTERNATIONAL MARKETS\nFor fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, compared to 60% \nand 61% for fiscal 2022 and fiscal 2021, respectively. We sell our products to retail accounts through our own NIKE Direct \noperations and through a mix of independent distributors, licensees and sales representatives around the world. W e sell to \nthousands of retail accounts and ship products from 67 distribution centers outside of the United States. Refer to Item 2. \nProperties for further information on distribution facilities outside of the United States. During fiscal 2023, NIKE's three largest \ncustomers outside of the United States accounted for approximately 14% of total non-U.S. sales.\nIn addition to NIKE-owned and Converse-owned digita","For fiscal 2023, non-U.S. NIKE Brand and Converse sales accounted for approximately 57% of total revenues, with thousands of retail accounts and 67 distribution centers outside of the United States."
2,"footwear production. For fiscal 2023, factories in Vietnam, Indonesia and China manufactured approximately 50%, 27% and 18% \nof total NIKE Brand footwear, respectively. For fiscal 2023, four footwear contract manufacturers each accounted for greater than \n10% of footwear production and in the aggregate accounted for approximately 58% of NIKE Brand footwear production.\nAs of May 31, 2023, our contract manufacturers operated 291 finished goods apparel factories located in 31 countries. For fiscal \n2023, NIKE Brand apparel finished goods were manufactured by 55 contract manufacturers, many of which operate multiple \nfactories. The largest single finished goods apparel factory accounted for approximately 8% of total fiscal 2023 NIKE Brand \napparel production. For fiscal 2023, factories in Viet","NIKE Brand footwear production in fiscal 2023 was mainly concentrated in Vietnam, Indonesia, and China, while the apparel production involved 55 contract manufacturers across 31 countries."
3,"of total NIKE Brand apparel, respectively. For fiscal 2023, one apparel contract manufacturer accounted for more than 10% of \napparel production, and the top five contract manufacturers in the aggregate accounted for approximately 52% of NIKE Brand \napparel production.\nNIKE's contract manufacturers buy raw materials for the manufacturing of our footwear, apparel and equipment products. Most \nraw materials are available and purchased by those contract manufacturers in the countries where manufacturing takes place. \nThe principal materials used in our footwear products are natural and synthetic rubber , plastic compounds, foam cushioning \nmaterials, natural and synthetic leather, nylon, polyester and natural fiber textiles, as well as polyurethane films used to make \nNIKE Air-Sole cushioning","NIKE's apparel production relies heavily on a small number of contract manufacturers, who also purchase raw materials for productio

Finally, we can save the output to a csv file.

In [12]:
output_dir = 'data/output'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

df.to_csv(f"{output_dir}/Nike_10k_Summaries.csv", index=False)